In [2]:
import pickle
import numpy as np
import tensorflow as tf
import math

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data

In [3]:
#load the CIFAR-10
def load_CIFAR10(pos, n_chunks=1):
    Xtr = []
    Ytr = []
    for i in range(n_chunks):
        train = unpickle(pos + '/data_batch_{0}'.format(i + 1))
        Xtr.extend(train[b'data'])
        Ytr.extend(train[b'labels'])
    test = unpickle(pos + '/test_batch')
    Xte = test[b'data']
    Yte = test[b'labels']
    return np.array(Xtr), np.array(Ytr), np.array(Xte), np.array(Yte)

In [4]:
# expresses the label data in one-hot encoding.
def onehot_encoding (Ytr, Yte):
    Ytr_onehot = np.eye(10)[Ytr]
    Yte_onehot = np.eye(10)[Yte]
    return Ytr_onehot, Yte_onehot

In [5]:
# load the train and test data
Xtr, Ytr, Xte, Yte = load_CIFAR10('cifar-10-batches-py', 5)
                                 
# image data, each data size is 32*32*3
Xtr = Xtr.reshape(50000, 3, 32, 32).transpose(0,2,3,1).astype("float")
Xte= Xte.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")

# label data of train and test data, label data is represented by one-hot encoding
Ytr, Yte = onehot_encoding(Ytr, Yte)

In [50]:
X = tf.placeholder(tf.float32, [None, 32, 32, 3])
Y = tf.placeholder(tf.float32, [None, 10])

In [ ]:
class CNN:
    def __init__(self, sess, beta, gamma, n_row):
        self.sess = sess
        sess.run(tf.global_variables_initializer())
        self.beta = beta
        self.gamma = gamma
        self.batch_size = None
        self.n_row = n_row
    def modeling(self, X):
        L1 = self.Layer(X, 7, 3, 64, 2, 'Layer_1') # filter 7x7x3x64, stride 2
        L1_pooling = self.pooling(L1, 3, 2)
        L2_1 = self.Layer(L1_pooling, 1, 64, 64, 1, 'Layer_2_1' pad='VALID')
        L2_2 = self.Layer(L2_1, 3, 64, 64, 1, 'Layer_2_2', pad='VALID')
        L2_3 = self.Layer(L2_2, 1, 64, 256, 1, 'Layer_2_3', pad='VALID')
        L2_4 = self.Layer(L1_pooling, 1, 64, 256, 1, 'Layer_2_4', pad='VALID')
        L2 = tf.nn.relu(L2_3 + L2_4)
        Layer = self.pooling(L2, 56, 1)
        return Layer
    def _batch(self, iterable, length, n=1):
        for ndx in range(0, length, n):
            yield iterable[ndx:min(ndx + n, length)]
    def train(self, X, y, n_epoch, batch_size):
        n_row = self.n_row
        sess = self.sess
        self.batch_size = batch_size
        n_step = n_row // batch_size
        loss = self.loss_funtion(X, y)
        for epoch in range(n_epoch):
            arr = np.random.randint(n_row, size=n_row)
            for x in self._batch(range(n_row), n_row, batch_size):
                batch_X, batch_y = X[arr[x], :], y[arr[x]]
                loss_ = sess.run(loss, feed_fict:{X:batch_X, y:batch_y})
                print(loss_)
#             for step in range(n_step):
#                 pass
    def loss_funtion(self, X, y, learning_rate=0.001):
        Layer = self.modeling(X)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Layer, labels=y))
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
        return loss
    def Layer(self, X, filter_size, in_feature, out_feature, stride, name, pad = 'SAME'):
        with tf.variable_scope(name):
            L = self.conv_bn_relu(X, filter_size, in_feature, out_feature, stride, pad = 'SAME')
        return L
    def conv_bn_relu(self, X, filter_size, in_feature, out_feature, stride, pad = 'SAME'):
        batch, beta, gamma = self.batch_size, self.beta, self.gamma
        stddev = math.sqrt(2/batch) if batch else 0.01
        W = tf.Variable(tf.random_normal([filter_size, filter_size, in_feature, out_feature], stddev=stddev))
        L = tf.nn.conv2d(X, W, strides=[1,stride,stride,1], padding=pad)
#         L = tf.layers.batch_normalization(L, training=True)
        L = tf.nn.batch_normalization(L, offset=beta, scale=gamma)
        L = tf.nn.relu(L)
        return L
    def pooling(self, L, ksize, stride):
        L = tf.nn.max_pool(L, ksize=[1, ksize, ksize, 1], strides=[1, stride, stride, 1], padding='SAME')
        return L
    def save_model(self):
        pass

In [51]:
### Implement the layers of CNNs ###

In [52]:
# cost function, you can change the implementation
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

In [53]:
# initialize the variables
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [54]:
### Implement the train process ###

In [55]:
### Implement the test process ###